Features to use:
    
   - genome length
   - number of rRNA operons
   - PFAM
   - TIGRFAMS
   
I am not going to use EC number because they are redundant with PFAM and TIGRFAM.

In [27]:
import os
import re

uniprotDir = './data/uniprotFiles/'
genomeDir = './data/genomeFiles/'
organisms = [x[:-4] for x in os.listdir(uniprotDir)]
print len(organisms)
myco_org = [x for x in organisms if "ycobac" in x]
print myco_org

605
['mycobacterium abscessus', 'mycobacterium chubuense', 'mycobacterium fortuitum', 'mycobacterium gilvum', 'mycobacterium hassiacum', 'mycobacterium marinum', 'mycobacterium neoaurum', 'mycobacterium novocastrense', 'mycobacterium obuense', 'mycobacterium rhodesiae', 'mycobacterium simiae', 'mycobacterium thermoresistibile', 'mycobacterium vanbaalenii']


In [4]:
# Testing 
org = "achromobacter xylosoxidans"
proteins = open(uniprotDir + org + '.txt').read()
t = re.findall(r'(TIGR\d+?);',proteins)
p = re.findall(r'(PF\d+?);',proteins)



### Extracting families

In [28]:
data = {x : {"functions" : [], "16S" : 0, "genomeLength" : 0} for x in organisms}

for f in os.listdir(uniprotDir):
#     print f
    uniprotFile = open(uniprotDir + f).read()
#     data[f[:-4]]["functions"] = data[f[:-4]]["functions"] + (re.findall(r'(TIGR\d\d\d\d\d);', uniprotFile))
    data[f[:-4]]["functions"] = data[f[:-4]]["functions"] + (re.findall(r'(PF\d\d\d\d\d);', uniprotFile))

### Extracting genome length and 16S operons

In [23]:
# Testing 
org = "mycobacterium gilvum"
proteins = open(genomeDir + org + '.gpff').read()
l = re.findall(r'LOCUS.+?(\d+) bp',proteins)
r = re.findall(r'rRNA  [\s\S\w\W]+?/product="([^"]+)',proteins)
r16 = len([x for x in r if "16" in x])
print r16



2


In [29]:
noRNA = []
numRNA = []

for f in organisms:
#     print f
    genomeFile = open(genomeDir + f + '.gpff').read()
#     print genomeDir + f + '.gpff'
    data[f]["genomeLength"] = sum([int(x) for x in re.findall(r'LOCUS.+?(\d+) bp',genomeFile)])
#     print re.findall(r'rRNA  [\s\S\w\W]+?/product="([^"]+)',genomeFile)
    rRNA = len([x for x in re.findall(r'rRNA  [\s\S\w\W]+?/product="([^"]+)',genomeFile) if "16" in x])
    if rRNA:
        data[f]["16S"] = rRNA
        if not rRNA in numRNA:
            numRNA.append(rRNA)

    else:
        print "\tno 16S"
        noRNA.append(f)

    
print numRNA  

	no 16S
	no 16S
	no 16S
	no 16S
	no 16S
	no 16S
	no 16S
	no 16S
	no 16S
[5, 2, 1, 3, 6, 4, 8, 7, 9, 10, 15, 64]


In [30]:
print len(noRNA)
with open('wrongGenome.txt', 'w') as f:
    f.write('\n'.join(noRNA))

9


### Formatting data

In [31]:
features = set()
for o in data.keys():
    if len(data[o]["functions"]) > 1:
        features = features.union(set(data[o]["functions"]))

print len(features)

14616


In [32]:
features = sorted(features)
organisms = sorted(organisms)

In [33]:
featuresInd = {}
organismsInd = {}
for i in range(len(features)):
    featuresInd[features[i]] = i

for j in range(len(organismsInd)):
    organismsInd[organisms[j]] = j

In [91]:
print features[0:10]
print featuresInd['PF00004']
# print featuresInd

['PF00001', 'PF00002', 'PF00003', 'PF00004', 'PF00005', 'PF00006', 'PF00007', 'PF00008', 'PF00009', 'PF00010']
3


In [34]:
formattedData = {x : {"features" : [0]*len(features), "16S" : 0, "genomeLength" : 0} for x in organisms}

for o in organisms:
    for f in data[o]["functions"]:
        formattedData[o]["features"][featuresInd[f]] += 1
    formattedData[o]["16S"] = data[o]["16S"]
    formattedData[o]["genomeLength"] = data[o]["genomeLength"]/1000000.0

In [51]:
# Saving data to file - 1

#Count for how many organisms don't have a 16S (to be removed from)
j = 0

# Data
with open('./formattedData/pfam_16S_length_1206.data','w') as f:
    for i in range(len(organisms)):
# Removing organisms without 16S
        if formattedData[organisms[i]]["16S"] != 0:
            if formattedData[organisms[i]]["16S"] > 10:
                line = [str(2)]
            else:
                line = [str(formattedData[organisms[i]]["16S"])]
            line = line + [str(formattedData[organisms[i]]["genomeLength"])]
            line = line + [str(x) for x in formattedData[organisms[i]]["features"]]
            f.write('\t'.join(line) + '\n')
        else:
            j = j + 1
print str(j) + " organisms don't have a 16S and have been removed from the data."

9 organisms don't have a 16S and have been removed from the data.


In [52]:
# Loading gr from all bacdive organisms
gr = {}
with open('./data/ncbi+bacdive_organisms_scaffold.singleGR.txt') as f:
    next(f)
    for l in f:
        l = l.strip('\n\r').split('\t')
        gr[l[0]] = l[1]

grInd = {"1-2" : 0, "2-3" : 1, "3-7" : 2, "8-14" : 3, "10-14" : 4, ">14" : 5 }

In [53]:
# Saving data to file - 2

# Labels
with open('./formattedData/gr_1206.data','w') as f:
    for i in range(len(organisms)):
        if formattedData[organisms[i]]["16S"] != 0:
            line = [ gr[organisms[i]]]
            f.write('\t'.join(line) + '\n')
# Organisms            
with open('./formattedData/organisms_1206.data','w') as f:
    for i in range(len(organisms)):
        if formattedData[organisms[i]]["16S"] != 0:
            f.write( organisms[i] + '\n')       
# Feature            
with open('./formattedData/features_1206.name', 'w') as f:
    f.write('16S\ngenomeLength\n' + '\n'.join(features))

In [56]:
# Creating a levelled gr file
gr_1206 = open('./formattedData/gr_1206.data').readlines()
gr_levelled_1206 = [str(grInd[x.strip('\n')]) for x in gr_1206]
org_1206 = open('./formattedData/organisms_1206.data').readlines()
org_1206 = [x.strip('\n') for x in org_1206]

print gr_1206[0:10]
print gr_levelled_1206[0:10]

# Saving to file
with open('./formattedData/gr_1206.levelled.data','w') as f:
    f.write('\n'.join(gr_levelled_1206))
        
with open('./formattedData/gr_organisms_1206.levelled.data', 'w') as f:
    for i in range(len(org_1206)):
        f.write(gr_levelled_1206[i] + '\t' + org_1206[i] + '\n')

['3-7\n', '3-7\n', '1-2\n', '2-3\n', '1-2\n', '3-7\n', '2-3\n', '3-7\n', '3-7\n', '3-7\n']
['2', '2', '0', '1', '0', '2', '1', '2', '2', '2']


In [48]:
for o in formattedData.keys():
    if formattedData[o]["16S"] > 10:
        print o + ' : '+ str(formattedData[o]["16S"])


pelosinus sp. : 15
streptomyces afghaniensis : 64
